# SEGFORMER MODELING & TRAINING

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import evaluate
from transformers import AutoImageProcessor, TFSegformerForSemanticSegmentation, create_optimizer
from transformers.keras_callbacks import KerasMetricCallback

/opt/homebrew/Caskroom/miniforge/base/envs/ComputerVision/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataFrame = pd.read_csv('Dataset/labels.csv')
dataFrame.head()

,Unnamed: 0,label_list
0,0,NaN
1,1,accessories
2,2,bag
3,3,belt
4,4,blazer


In [4]:
ID2LABEL = {}
for i, j in dataFrame.iterrows():
    if i==0:
        ID2LABEL[i] = 'nan'
    else:
        ID2LABEL[i] = j['label_list']
ID2LABEL

{0: 'nan',
 1: 'accessories',
 2: 'bag',
 3: 'belt',
 4: 'blazer',
 5: 'blouse',
 6: 'bodysuit',
 7: 'boots',
 8: 'bra',
 9: 'bracelet',
 10: 'cape',
 11: 'cardigan',
 12: 'clogs',
 13: 'coat',
 14: 'dress',
 15: 'earrings',
 16: 'flats',
 17: 'glasses',
 18: 'gloves',
 19: 'hair',
 20: 'hat',
 21: 'heels',
 22: 'hoodie',
 23: 'intimate',
 24: 'jacket',
 25: 'jeans',
 26: 'jumper',
 27: 'leggings',
 28: 'loafers',
 29: 'necklace',
 30: 'panties',
 31: 'pants',
 32: 'pumps',
 33: 'purse',
 34: 'ring',
 35: 'romper',
 36: 'sandals',
 37: 'scarf',
 38: 'shirt',
 39: 'shoes',
 40: 'shorts',
 41: 'skin',
 42: 'skirt',
 43: 'sneakers',
 44: 'socks',
 45: 'stockings',
 46: 'suit',
 47: 'sunglasses',
 48: 'sweater',
 49: 'sweatshirt',
 50: 'swimwear',
 51: 't-shirt',
 52: 'tie',
 53: 'tights',
 54: 'top',
 55: 'vest',
 56: 'wallet',
 57: 'watch',
 58: 'wedges'}

In [5]:
LABEL2ID = {v: k for k, v in ID2LABEL.items()}
LABEL2ID

{'nan': 0,
 'accessories': 1,
 'bag': 2,
 'belt': 3,
 'blazer': 4,
 'blouse': 5,
 'bodysuit': 6,
 'boots': 7,
 'bra': 8,
 'bracelet': 9,
 'cape': 10,
 'cardigan': 11,
 'clogs': 12,
 'coat': 13,
 'dress': 14,
 'earrings': 15,
 'flats': 16,
 'glasses': 17,
 'gloves': 18,
 'hair': 19,
 'hat': 20,
 'heels': 21,
 'hoodie': 22,
 'intimate': 23,
 'jacket': 24,
 'jeans': 25,
 'jumper': 26,
 'leggings': 27,
 'loafers': 28,
 'necklace': 29,
 'panties': 30,
 'pants': 31,
 'pumps': 32,
 'purse': 33,
 'ring': 34,
 'romper': 35,
 'sandals': 36,
 'scarf': 37,
 'shirt': 38,
 'shoes': 39,
 'shorts': 40,
 'skin': 41,
 'skirt': 42,
 'sneakers': 43,
 'socks': 44,
 'stockings': 45,
 'suit': 46,
 'sunglasses': 47,
 'sweater': 48,
 'sweatshirt': 49,
 'swimwear': 50,
 't-shirt': 51,
 'tie': 52,
 'tights': 53,
 'top': 54,
 'vest': 55,
 'wallet': 56,
 'watch': 57,
 'wedges': 58}

In [6]:
NUM_CLASSES = len(ID2LABEL)
NUM_CLASSES

59

In [7]:
H,W = 512,512

In [8]:
modelID = "nvidia/segformer-b5-finetuned-ade-640-640"
model = TFSegformerForSemanticSegmentation.from_pretrained(modelID)
model.summary()

2025-02-08 16:20:35.423373: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-08 16:20:35.423431: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-08 16:20:35.423438: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1739060435.423463 10057117 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1739060435.423505 10057117 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFSegformerForSemanticSegmentation.

All the layers of TFSegformerForSemanticSegmentation were initialized from the model checkpoint at nvidia/segformer-b5-finetuned-ade-640-640.
If your task is similar t

Model: "tf_segformer_for_semantic_segmentation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 segformer (TFSegformerMain  multiple                  81443008  
 Layer)                                                          
                                                                 
 decode_head (TFSegformerDe  multiple                  3267222   
 codeHead)                                                       
                                                                 
Total params: 84710230 (323.14 MB)
Trainable params: 84708694 (323.14 MB)
Non-trainable params: 1536 (6.00 KB)
_________________________________________________________________


In [9]:
trainDataset = tf.data.Dataset.load('SavedDataset/trainDataset')
valDataset = tf.data.Dataset.load('SavedDataset/valDataset')

In [10]:
BATCH_SIZE = 2
NUM_EPOCHS = 15
NUM_TRAIN_STEPS = len(trainDataset)*NUM_EPOCHS
LEARNING_RATE = 6e-5
WEIGHT_DECAY = 0.01

optimizer, LR_Schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_train_steps=NUM_TRAIN_STEPS,
    weight_decay_rate=WEIGHT_DECAY,
    num_warmup_steps=0,
)

In [11]:
metrics = evaluate.load("mean_iou")

In [12]:
def computeMetrics(pred):
    logits, labels = pred
    logtis = tf.transpose(logits, (0, 2, 3, 1))
    logits = tf.image.resize(logtis, tf.shape(labels)[1:], method='bilinear')
    predLabels = tf.argmax(logits, axis=-1)

    metrics = metrics.compute(predictions=predLabels, references=labels, num_classes=NUM_CLASSES, ignore_index=-1)

    perCategoryAccuracy = metrics.pop('per_category_accuracy').toList()
    perCatehgoryIOU = metrics.pop('per_category_iou').toList()
    metrics.update({f'accuracy_{ID2LABEL[i]}': v for i, v in enumerate(perCategoryAccuracy)})
    metrics.update({f'iou_{ID2LABEL[i]}': v for i, v in enumerate(perCatehgoryIOU)})

    return {'val_' + k: v for k, v in metrics.items()}


In [13]:
metricsCallback = KerasMetricCallback(
    metric_fn=computeMetrics,
    eval_dataset=valDataset,
    batch_size=BATCH_SIZE,
    label_cols=['labels'],
)
callbacks = [metricsCallback]

In [14]:
model.compile(optimizer=optimizer)

In [ ]:
with tf.device('/CPU:0'):
    model.fit(trainDataset, validation_data=valDataset, epochs=NUM_EPOCHS, callbacks=callbacks)

Epoch 1/15


2025-02-08 16:21:03.368312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
I0000 00:00:1739060472.324206 10057252 service.cc:148] XLA service 0x421c56550 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739060472.324329 10057252 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1739060472.355907 10057252 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


186/462 [===========>..................] - ETA: 1:04:35 - loss: 0.2750

In [ ]:
model.save('Models/Segformer.keras')